# Epidemiologické modely

*KMA/VPM2*

*Jan Půlpán*

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from scipy.integrate import ode, solve_ivp
import matplotlib.pyplot as plt
from ipywidgets import interactive
import ipywidgets as widgets
from IPython.display import Markdown, display

plt.style.use("ggplot")

## Základní SIR model

Model rozděluje populaci do tří přihrádek (compartments):
- $S$ z anglického Susceptible - náchylný, myšleno k nákaze,
- $I$ jako Infectious - infekční, 
- $R$ jako Removed - odstraněný, myšleno již neinfekční.

Jednotlivé přihrádky jsou svázány pomocí soustavy 3 direfenciálních rovnic.

\begin{align*}
\frac{dS}{dt} &= - \frac{\beta I S}{N},\\
\frac{dI}{dt} &= \frac{\beta I S}{N} - \gamma I,\\
\frac{dR}{dt} &= \gamma I.\\
\end{align*}

Celkovou velikost populace značíme N, $\beta$ je koeficient tempa růstu nakažených a $\gamma$ je koeficient tempa uzdravení. V našem případě, kdy vztah udávající přírůstek infekčních dělíme velikostí populace $N$, udává $\beta$ průměrné množství osobních kontaktů za jednotku času (obvykle jeden den). Koeficient tempa zotavení $\gamma$ pak odpovídá průměrnému počtu dní, kdy je jeden člen populace infekční (značíme $D$), platí $\gamma = \frac{1}{D}$. 

U SIR modelu platí, že velikost populace $N$ je konstantní. Z rovnic je zřejmé, že $\frac{dN}{dt} = \frac{dS}{dt}+\frac{dI}{dt}+\frac{dR}{dt}=0$ a tedy $N=S(t)+I(t)+R(t)$.

Jestli vypukne epidemie, která se mezi populací šíří, záleží na tom, zda bude přibývat infekčních jedinců, $\frac{dI}{dt} > 0$. Z tohoto vztahu lze číslo $R_0 = \frac{\beta}{\gamma}$. To udává počet jedinců, které infekční jedinec nakazí. Pomocí $R_O$ lze získat i informace o průběhu epidemie, včetně maximálního počtu nakažených (vrchol epidemie)

$$I_{max}=S(0)+I(0)+ \frac{N}{R_0} \left( \ln{\frac{N}{R_0}}-\ln{S(0)}-1 \right),$$

nebo počet náchylných jedinců po skončení epidemie $S_{\infty}$, jako řešní následující rovnice

$$\ln{ \frac{S_{\infty}}{S_0}} = R_0\left(1-\frac{S_{\infty}}{N}\right).$$

Stacionární stav systému lze pak vyjádřit jako $(S_{\infty}, 0 , R_{\infty})$, kde $R_{\infty} = N - S_{\infty}$. Systém má obecně jen jeden stacionární stav, který je pro nulovou $I$ složku značíme DFE (z anglického desease free equilibrium).

In [2]:
N = 1000        # velikost populace
R0 = 0          # počáteční podmínka pro R
I0 = 10         # počáteční podmínka pro I
S0 = N-I0-R0    # počáteční podmínka pro S
max_time = 400  # maximální ča, pro který model počítat

# definice modelu
def sir_ode(times,ivs,params):
    beta, gamma = params
    S,I,R = ivs
    N = S+I+R
    # ODEs
    dS = -beta*S*I/N
    dI = beta*S*I/N-gamma*I
    dR = gamma*I
    return [dS,dI,dR]


def sir_solve(beta, gamma):
    fig = plt.figure(figsize=(35,12))
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)

    params = [beta, gamma]
    ivs = [S0,I0,R0]
    time = np.linspace(0,max_time,801)

    sir_sol = solve_ivp(fun=lambda t, y: sir_ode(t, y, params), 
                    t_span=[min(time),max(time)], 
                    y0=ivs, 
                    t_eval=time)

    t_sol = sir_sol['t']
    s_sol = sir_sol['y'][0]
    i_sol = sir_sol['y'][1]
    r_sol = sir_sol['y'][2]


    splot = ax1.plot(t_sol, s_sol, color='blue', linewidth=2, label='S')
    iplot = ax1.plot(t_sol, i_sol, color='red', linewidth=2, label='I')
    rplot = ax1.plot(t_sol, r_sol, color='green', linewidth=2, label='R')
    
    display(Markdown(r'$R_O$ = '+'{:.2f}'.format(beta/gamma)))
    display(Markdown(r'$D$ = '+'{:.2f}'.format(1/gamma)))
    
    plt.xlabel("t",fontweight="bold")
    legend = ax1.legend(loc=5,bbox_to_anchor=(1.1,0.5))
    frame = legend.get_frame()
    frame.set_facecolor("white")

    pal = ["#AA0000", "#00AA00", "#0000AA"]
    ax2.stackplot(t_sol,i_sol,r_sol,s_sol,colors=pal, alpha=0.5)
    plt.show()
    
interactive_plot = interactive(sir_solve, 
                               beta = widgets.FloatSlider(min=0, max=1, step=.05, value=0.1, readout_format='.2f'), 
                               gamma = widgets.FloatSlider(min=.005, max=.2, step=.01, value=0.05, readout_format='.3f'))
output = interactive_plot.children[-1]
interactive_plot

interactive(children=(FloatSlider(value=0.1, description='beta', max=1.0, step=0.05), FloatSlider(value=0.05, …

## SIRS model s vitální dynamikou


SIR model rozděluje populaci do tří přihrádek (compartments):
- $S$ z anglického Susceptible - náchylný, myšleno k nákaze,
- $I$ jako Infectious - infekční, 
- $R$ jako Removed - odstraněný, myšleno již neinfekční.

Základní SIR model počítá s tím, že již neinfekční jedinec zůstane v přihrádce Removed, nemůže se znovu stát náchylným k nákaze. Jedna z možností jak SIR model trochu přiblížit realitě je zahrnout právě opačné chování. Takový model se nazývá SIRS a počítá s tím, že získaná imunita (přihrádka R) není na vždy, ale někteří již neinfekční a odstranění jedinci se znovu stávají náchylnými (přihrádka $S$). 

Jednoduchý SIR model neuvažuje také natalitu $\Lambda$ (počet nově narozených za rok) a mortalitu $\mu$ (podíl zemřelých za rok) protože je epidemie většinou dostatečně rychlá na to, aby dynamika těchto procesů mohla být zanedbána. My oba parametry ale zahrneme do SIRS modelu, mortalitu navíc nastavíme zvlášť pro jednotlivé S,I a R přihrádky. Stejně jako v SIR modelu zachováme konstatní velikost populace $N$. Mortalita bude plně kompenzována natalitou, $\Lambda = \mu_1 S + \mu_2 I + \mu_3 R$.

Náš SIRS model vydadá takto:

\begin{align*}
\frac{dS}{dt} &= \Lambda - \frac{\beta I S}{N} - \mu_1 S + \nu R,\\
\frac{dI}{dt} &= \frac{\beta I S}{N} - \mu_2 I - \gamma I,\\
\frac{dR}{dt} &= \gamma I - (\mu_3 + \nu) R.\\
\end{align*}

Celkovou velikost populace značíme N, $\beta$ je koeficient tempa růstu nakažených a $\gamma$ je koeficient tempa uzdravení. V našem případě, kdy vztah udávající přírůstek infekčních dělíme velikostí populace $N$, udává $\beta$ průměrné množství osobních kontaktů za jednotku času (obvykle jeden den). Koeficient tempa zotavení $\gamma$ pak odpovídá průměrnému počtu dní, kdy je jeden člen populace infekční (značíme $D_I$), platí $\gamma = \frac{1}{D_I}$. 

Oproti základnímu SIR modelu se v SIRS objevuje parametr tempa ztráty imunity $\nu$, který odpovídá průměrnému počtu dní $D_R$, po kterých jedinec ztratí imunitu $\nu = \frac{1}{D_R}$.

I u našeho SIRS modelu platí, že velikost populace $N$ je konstantní. Pokud uvažujeme že mortalita je kompenzována natalitou, pak je z rovnic zřejmé $\frac{dN}{dt} = \frac{dS}{dt}+\frac{dI}{dt}+\frac{dR}{dt}=0$ a tedy $N=S(t)+I(t)+R(t)$.



Protože máme konstatní populaci a natalita kompenzuje mortalitu, rovnice můžeme upravit do tvaru

\begin{align*}
\frac{dS}{dt} &= - \frac{\beta I S}{N} + \mu_2 I + (\mu_3 +\nu) R,\\
\frac{dI}{dt} &= \frac{\beta I S}{N} - (\mu_2 + \gamma) I,\\
\frac{dR}{dt} &= \gamma I - (\mu_3 + \nu) R.\\
\end{align*}

Reprodukční číslo $R_0 = \frac{\beta}{\mu_2 + \gamma}$ odvodíme stejným způsobem, jako u SIR modelu.

In [81]:
N = 1000        # velikost populace
R0 = 0          # počáteční podmínka pro R
I0 = 10         # počáteční podmínka pro I
S0 = N-I0-R0    # počáteční podmínka pro S
max_time = 500  # maximální ča, pro který model počítat

def sirs_ode(t,ivs,params):
    beta, gamma, mu2, mu3, nu = params
    S,I,R = ivs
    N = S+I+R
    # ODEs
    dS = -beta*S*I/N + mu2*I + (mu3+nu)*R
    dI = beta*S*I/N-(mu2+gamma)*I
    dR = gamma*I-(mu3+nu)*R
    return [dS,dI,dR]


def sirs_solve(beta, gamma, mu_2, mu_3, nu):
    
    display(Markdown(r'$R_O$ = '+'{:.2f}'.format(beta/(mu_2+gamma))))
    display(Markdown(r'$D_I$ = '+'{:.2f}'.format(1/gamma)))
    display(Markdown(r'$D_R$ = '+'{:.2f}'.format(1/nu)))
    
    fig = plt.figure(figsize=(32,12))
    ax1 = fig.add_subplot(221)
    ax2 = fig.add_subplot(223)
    ax3 = fig.add_subplot(122)


    # soustava diferencialnich rovnic
    def f(t, y):
        y1, y2 = y
        return [-beta*y1*y2 + (mu_2-mu_3-nu)*y2 - (mu_3+nu)*y1 + mu_3+nu,
                beta*y1*y2 - (mu_2+gamma)*y2]


    # nalezeni klidovych stavu
    ys1, ys2 = symbols('ys1, ys2', real=True)
    stable = nonlinsolve([-beta*ys1*ys2 + (mu_2-mu_3-nu)*ys2 - (mu_3+nu)*ys1 + mu_3+nu,
                          beta*ys1*ys2 - (mu_2+gamma)*ys2], [ys1, ys2])
    for i in range(len(stable.args)):
        display(Markdown('Stacionární bod '+str(i+1)+': $({:.3f},{:.3f})$'.format(stable.args[i][0],float(stable.args[i][1]))))

      
    # vykresleni vektoroveho pole
    y1, y2 = np.meshgrid(np.linspace(-.1, 1.1, 20), np.linspace(-.1, 1.1, 20))
    u, v = np.zeros(y1.shape), np.zeros(y2.shape)
    m, n = y1.shape
    t = 0
    for i in range(m):
        for j in range(n):
            u[i,j], v[i,j] = f(t, [y1[i, j], y2[i, j]])

    ax3.quiver(y1, y2, u, v, color='lightgray')

    # vykresleni klidovych stavu a fazoveho portretu
    for s in range(len(stable.args)):
            for r in np.linspace(.05,1.2,10):
                for phi in np.linspace(0, 2*np.pi, int(np.random.rand()*10+6)):
                    sol = integrate.solve_ivp(f,[0,250], 
                                              [r*np.cos(phi)+stable.args[s][0],
                                               r*np.sin(phi)+stable.args[s][1]], 
                                               t_eval=np.linspace(0,250,1000))
                    ax3.plot(sol.y[0], sol.y[1], 'k-')
                    ax3.plot([sol.y[0][0]], [sol.y[1][0]], 'ko')


    for s in range(len(stable.args)):
        if stable.args[s][0].is_real and stable.args[s][1].is_real:
            ax3.plot(stable.args[s][0], stable.args[s][1], marker=".", color='red', markersize=30)
    
    # phase portrait - domain triangle
    tri = Polygon(((0.,0.), (1.,0.), (0., 1.)),
                    fc=(0,0,1,0.18))
    ax3.add_artist(tri)
    
    
    ax3.set_xlim([-.03, 1.03])
    ax3.set_ylim([-.03, 1.03])
    ax3.set_xlabel('$S$')
    ax3.set_ylabel('$I$')
    
    
    params = [beta, gamma, mu_2, mu_3, nu]
    ivs = [S0,I0,R0]
    time = np.linspace(0,max_time,801)

    sirs_sol = solve_ivp(fun=lambda t, y: sirs_ode(t, y, params), 
                    t_span=[min(time),max(time)], 
                    y0=ivs, 
                    t_eval=time)

    t_sol = sirs_sol['t']
    s_sol = sirs_sol['y'][0]
    i_sol = sirs_sol['y'][1]
    r_sol = sirs_sol['y'][2]


    splot = ax1.plot(t_sol, s_sol, color='blue', linewidth=2, label='S')
    iplot = ax1.plot(t_sol, i_sol, color='red', linewidth=2, label='I')
    rplot = ax1.plot(t_sol, r_sol, color='green', linewidth=2, label='R')
    

    ax2.set_xlabel("t",fontweight="bold")
    legend = ax1.legend(loc=5,bbox_to_anchor=(1.1,0.5))
    frame = legend.get_frame()
    frame.set_facecolor("white")

    pal = ["#AA0000", "#00AA00", "#0000AA"]
    ax2.stackplot(t_sol,i_sol,r_sol,s_sol,colors=pal, alpha=0.5)
    plt.show()
    
interactive_plot = interactive(sirs_solve, 
                               beta = widgets.FloatSlider(min=0, max=1, step=.05, value=0.1), 
                               gamma = widgets.FloatSlider(min=.01, max=.3, step=.01, value=0.05),
                               mu_2 = widgets.FloatSlider(min=0, max=.02, step=.005, value=0.01, readout_format='.3f'),
                               mu_3 = widgets.FloatSlider(min=0, max=0.01, step=0.002, value=0.01, readout_format='.3f'),
                               nu = widgets.FloatLogSlider(min=-3, 
                                                           max=0, 
                                                           base=10, 
                                                           step=.02, 
                                                           value=0.01, 
                                                           description='nu'))
output = interactive_plot.children[-1]

interactive_plot

interactive(children=(FloatSlider(value=0.1, description='beta', max=1.0, step=0.05), FloatSlider(value=0.05, …


Pokusit se zjisti něco o stacionárních stavech:
- jak je to ze standardním SIR modelem? - tam je snad jen $(S^*,0,R^*)$ 
- jak je to s modelem s vitální dynamikou? - tam kromě tohodle stavu přibejvá i  $(S^*,I^*,R^*)$ - jak je to ale závislé na parametrech? (tohle asi nejsem schopen udělat, pokud těch parametrů mám víc jak 2

- DFE - je asymptoticky stabilní, pokud je $R_0 < 1$, pro $R_0>1$ je nestabilní.
- Endemic Equilibrium (EE): $(\bar{S}, \bar{I}, \bar{R})$ kde $\bar{R} = N - \bar{S} - \bar{I}$

$$\bar{S}=\frac{1}{R_0}$$

$$\bar{I}=\frac{\mu_3}{\mu_3+\gamma}\left(1-\frac{1}{R_0}\right)$$
    - EE je asymtomaticky stabilní, pokud $R_0>1$


**Závěry:**
- pokud je $\nu=0$ (máme tedy SIR model) tak se projeví mortalita ... ???
- protože je mortalita kompenzovaná natalitou, tak se mi vůbec neprojevuje, nebo jen v extrémně vysokých případech mortality
- předchozí neni pravda, projevuje se to, pokud je nu nízké (imunita se ztrácí za dlouho)


- díky tomu, že se vrací R ––> S, vzniká mi endemický stacionární stav

- měnit mortalitu na I nedává smysl, protožej je kompenzovaná natalitou a to neodpovídá skutečnosti

Zdroje:
- Wikipedia: https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology#The_SIR_model
- Philippe Adda, Derdei Bichara. Global stability for SIR and SIRS models with differential mortality.International Journal of Pure and Applied Mathematics, Academic Publishing Ltd, 2012, GLOBALSTABILITY FOR SIR AND SIRS MODELS WITH DIFFERENTIAL MORTALITY, 80 (3), pp.425-433. hal-00675359v2

In [61]:
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt
from sympy.core.symbol import symbols
from sympy.solvers.solveset import nonlinsolve
from sympy import init_printing
from IPython.display import Markdown, display
from ipywidgets import interactive
import ipywidgets as widgets

init_printing(use_latex='mathjax')

plt.rcParams['figure.figsize'] = [17, 10]


def sirs_pd(beta, gamma, mu_2, mu_3, nu):
    # soustava diferencialnich rovnic
    def f(t, y):
        y1, y2 = y
        return [-beta*y1*y2 + (mu_2-mu_3-nu)*y2 - (mu_3+nu)*y1 + mu_3+nu,
                beta*y1*y2 - (mu_2+gamma)*y2]


    # nalezeni klidovych stavu
    ys1, ys2 = symbols('ys1, ys2', real=True)
    stable = nonlinsolve([-beta*ys1*ys2 + (mu_2-mu_3-nu)*ys2 - (mu_3+nu)*ys1 + mu_3+nu,
                          beta*ys1*ys2 - (mu_2+gamma)*ys2], [ys1, ys2])
    for i in range(len(stable.args)):
        display(Markdown('Stacionární bod '+str(i+1)+': $('+str(stable.args[i][0])+','+str(stable.args[i][1])+')$'))

        
        
    fig, ax = plt.subplots()
    
    
    # vykresleni vektoroveho pole
    y1, y2 = np.meshgrid(np.linspace(-.1, 1.1, 20), np.linspace(-.1, 1.1, 20))
    u, v = np.zeros(y1.shape), np.zeros(y2.shape)
    m, n = y1.shape
    t = 0
    for i in range(m):
        for j in range(n):
            u[i,j], v[i,j] = f(t, [y1[i, j], y2[i, j]])

    plt.quiver(y1, y2, u, v, color='lightgray')

    # vykresleni klidovych stavu a fazoveho portretu
    for s in range(len(stable.args)):
            for r in np.linspace(.05,1.2,10):
                for phi in np.linspace(0, 2*np.pi, int(np.random.rand()*10+6)):
                    sol = integrate.solve_ivp(f,[0,250], 
                                              [r*np.cos(phi)+stable.args[s][0],
                                               r*np.sin(phi)+stable.args[s][1]], 
                                               t_eval=np.linspace(0,250,1000))
                    plt.plot(sol.y[0], sol.y[1], 'k-')
                    plt.plot([sol.y[0][0]], [sol.y[1][0]], 'ko')


    for s in range(len(stable.args)):
        if stable.args[s][0].is_real and stable.args[s][1].is_real:
            plt.plot(stable.args[s][0], stable.args[s][1], marker=".", color='red', markersize=30)

            
   
    tri = Polygon(((0.,0.), (1.,0.), (0., 1.)),
                    fc=(0,0,1,0.2))
    ax.add_artist(tri)
    
    display(Markdown(r'$R_O$ = '+'{:.2f}'.format(beta/(mu_2+gamma))))
    display(Markdown(r'$D_I$ = '+'{:.2f}'.format(1/gamma)))
    display(Markdown(r'$D_R$ = '+'{:.2f}'.format(1/nu)))
            
    plt.xlim([-.03, 1.03])
    plt.ylim([-.03, 1.03])
    plt.xlabel('$U$')
    plt.ylabel('$V=U\'$')
    plt.show()
    
    

interactive_plot = interactive(sirs_pd, 
                               beta = widgets.FloatSlider(min=0, max=1, step=.05, value=0.1), 
                               gamma = widgets.FloatSlider(min=.01, max=.3, step=.01, value=0.05),
                               mu_2 = widgets.FloatSlider(min=0, max=.02, step=.005, value=0.01, readout_format='.3f'),
                               mu_3 = widgets.FloatSlider(min=0, max=0.01, step=0.002, value=0.01, readout_format='.3f'),
                               nu = widgets.FloatLogSlider(min=-3, 
                                                           max=0, 
                                                           base=10, 
                                                           step=.02, 
                                                           value=0.01, 
                                                           description='nu'))
output = interactive_plot.children[-1]

interactive_plot

interactive(children=(FloatSlider(value=0.1, description='beta', max=1.0, step=0.05), FloatSlider(value=0.05, …